# 2.2.1 텐서 다루기

In [1]:
pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
print(torch.__version__)

1.12.1+cu113


## 텐서 생성 및 변환

In [10]:
import torch
print(torch.tensor([[1,2],[3,4]])) #2차원 행태의 텐서 생성
print(torch.tensor([[1,2],[3,4]], device="cuda:0")) #GPU에 텐서 생성
print(torch.tensor([[1,2],[3,4]], dtype=torch.float64)) #dtype을 이용하여 텐서 생성

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]], device='cuda:0')
tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)


In [11]:
temp = torch.tensor([[1,2],[3,4]])
print(temp.numpy())

temp = torch.tensor([[1,2],[3,4]], device="cuda:0")
print(temp.to("cpu").numpy())

[[1 2]
 [3 4]]
[[1 2]
 [3 4]]


## 텐서의 인덱스 조작

torch.FloatTensor : 32비트의 부동 소수점<br>
torch.DoubleTensor : 64비트의 부동 소수점<br>
torch.LongTensor : 64비트의 부호가 있는 정수

In [12]:
temp = torch.FloatTensor([1,2,3,4,5,6,7])
print(temp[0], temp[1], temp[-1])
print("--------------------")
print(temp[2:5], temp[4:-1])

tensor(1.) tensor(2.) tensor(7.)
--------------------
tensor([3., 4., 5.]) tensor([5., 6.])


## 텐서 연산 및 차원 조작

In [13]:
v = torch.tensor([1,2,3])
w = torch.tensor([3,4,6])
print(w-v)

tensor([2, 2, 3])


In [16]:
temp = torch.tensor([[1,2],[3,4]])

print(temp.shape)
print("--------------------")
print(temp.view(4,1))
print("--------------------")
print(temp.view(-1))
print("--------------------")
print(temp.view(1,-1))
print("--------------------")
print(temp.view(-1,1))
print("--------------------")
print(temp.view(-1, 2))

torch.Size([2, 2])
--------------------
tensor([[1],
        [2],
        [3],
        [4]])
--------------------
tensor([1, 2, 3, 4])
--------------------
tensor([[1, 2, 3, 4]])
--------------------
tensor([[1],
        [2],
        [3],
        [4]])
--------------------
tensor([[1, 2],
        [3, 4]])


## 2.2 데이터 준비

In [18]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

## 커스텀 데이터 셋 구현 방법

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, csv_file): # csv_file 파라키터를 통해 데이터셋 불러오기
    self.label = pd.read_csv(csv_file)

  def __len__(self): # 전체 데이터셋 크기 반환
    return len(self.label)

  def __getitem__(self, idx): # 전체x와 y 데이터 중에 해당 idx번째의 데이터를 가져온다.
    sample = torch.tensor(self.label.iloc[idx,0:3]),int()
    label = torch.tensor(self.label.iloc[idx,3]).int()
    return sample, label

tensor_dataset = CustomDataset('../covtype.csv') # 괄호안에 데이터셋 넣기
dataset = DataLoader(tensor_dataset, batch_size=4, shuffle=True) # 모델을 보관하고 있다가 배치크기만큼 데이터를 꺼내서 사용

## 2.2.3 모델 정의

### Sequential 신경망을 정의하는 방법

In [29]:
import torch.nn as nn
class MLP(nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2))
    
    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels=64, out_channels=30, kernel_size=5),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2))
    
    self.layer3 = nn.Sequential(
        nn.Linear(in_features = 30*5*5, out_features=10, bias=True), 
        nn.ReLU(inplace=True))

    def forward(self, x):
      x = self.layer1(x)
      x = self.layer2(x)
      x = x.view(x.shape[0], -1)
      x = self.layer3(x)
      return x
  
model = MLP()

print("Printing children\n-------------")
print(list(model.children()))
print("\n\nPrinting Modules\n-------------")
print(list(model.modules()))

Printing children
-------------
[Sequential(
  (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
), Sequential(
  (0): Conv2d(64, 30, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
), Sequential(
  (0): Linear(in_features=750, out_features=10, bias=True)
  (1): ReLU(inplace=True)
)]


Printing Modules
-------------
[MLP(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(64, 30, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=750, out_features=10